In [1]:
#import relevant packages and libraies
import os
import re
import cv2
import numpy as np
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import tensorflow.keras as keras
from keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import models, layers
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from keras.models import Model

from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay


In [2]:
situations = {
                'c0': 'Safe driving', 
                'c1': 'Texting - right', 
                'c2': 'Talking on the phone - right', 
                'c3': 'Texting - left', 
                'c4': 'Talking on the phone - left', 
                'c5': 'Operating the radio', 
                'c6': 'Drinking', 
                'c7': 'Reaching behind', 
                'c8': 'Hair and makeup', 
                'c9': 'Talking to passenger'
}

In [3]:
X = []
y = []
classes = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
base_path = '../input/state-farm-distracted-driver-detection/imgs/train/'

for i, target in enumerate(classes):
    
    files = os.listdir(base_path+target)
    
    print(f'we are now in {target} class')
    
    for file in files:
        
        # load the image
        img = load_img(base_path+target + '/' + file, target_size=(224, 224))
        
        #convert it to an array
        img_array = np.array(img)
              
        # append the array to X
        X.append(img_array)
        
        # append the numeric target to y
        y.append(i) 
        
print('finished')

In [4]:
X = np.array(X)
y = np.array(y)

In [5]:
# shuffle the data
shuffler = np.random.permutation(len(X))
X = X[shuffler]
y = y[shuffler]

In [6]:
# train-test split
from sklearn.model_selection import train_test_split

X_train_sample, X_test_sample, y_train_sample, y_test_sample = train_test_split(X, y, test_size=0.90)

In [7]:
X_train,X_test, y_train, y_test = train_test_split(X_train_sample,y_train_sample, test_size=0.20)

In [8]:
#keep a copy of y test as the actual number labels for further process
y_test_true = y_test.copy()

In [9]:
#one hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [10]:
#scaling
X_train = X_train.astype('float32')/255

In [11]:
X_test = X_test.astype('float32')/255

In [12]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

# CNN

In [13]:
#clear session 
K.clear_session()

In [14]:
# number of possible label values
nb_classes = 10

#designing the layers and build model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(224,224,3),padding = 'same'))
model.add(BatchNormalization())

#model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(224,224,3),padding = 'same'))
#model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64,kernel_size = (3,3),activation = 'relu',padding = 'same'))
model.add(BatchNormalization())

#model.add(Conv2D(64,kernel_size = (3,3),activation = 'relu',padding = 'same'))
#model.add(BatchNormalization())

model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64,kernel_size = (3,3),activation = 'relu',padding = 'same'))
model.add(BatchNormalization())


#model.add(Conv2D(64,kernel_size = (3,3),activation = 'relu',padding = 'same'))
#model.add(BatchNormalization())


model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128,kernel_size = (3,3),activation = 'relu',padding = 'same'))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))
model.add(Dense(nb_classes, activation='softmax'))

In [15]:
#check the summary:
model.summary()

In [16]:
#compile the model
model.compile(optimizer=SGD(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
#fit the model
history = model.fit(X_train, y_train, batch_size=50, epochs=50, validation_split=0.25)

In [19]:
model.save('model_cnn_sgd_scaling.h5')

In [20]:
ypred = model.predict(X_test)

In [21]:
from pylab import rcParams
rcParams['figure.figsize'] = 12,20
# have a look at the data
for i in range(10):
    plt.subplot(5, 2, i+1)
    plt.imshow(X_test[i])
    plt.axis('off')
    plt.title(situations[classes[np.argmax(ypred[i])]]) 

# Examine the model

In [22]:
#plot accuracy curve
plt.figure(figsize=(8,6))
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label= 'validation accuracy')

plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()


In [23]:
# Plot loss
plt.figure(figsize=(8,6))
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

In [24]:
true_list =[]
for i in range(len(y_test_true)):
    if np.argmax(ypred[i])==y_test_true[i]:
        true_list.append(i)
        
probability = len(true_list) / len(y_test_true)
round(probability,3)

In [25]:
#plot confusion matrix
cm = confusion_matrix(y_true=y_test_true, y_pred=np.argmax(ypred, axis=1))

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=classes)
fig, ax = plt.subplots(figsize=(10,10))

disp.plot(ax=ax);

In [26]:
from pylab import rcParams
rcParams['figure.figsize'] = 12,20
# have a look at the data
for i in range(10):
    plt.subplot(5, 2, i+1)
    plt.imshow(X_test[i])
    plt.axis('off')
    plt.title(situations[classes[np.argmax(ypred[i])]]) 

In [27]:
history.history

In [ ]:
history_x =  {'loss': [2.1738808155059814,
  1.1549928188323975,
  0.6281741261482239,
  0.3763605058193207,
  0.23964114487171173,
  0.16851677000522614,
  0.1265759915113449,
  0.09741301834583282,
  0.07863800972700119,
  0.06945163011550903,
  0.05451241135597229,
  0.048303086310625076,
  0.04314027354121208,
  0.0400322824716568,
  0.036266081035137177,
  0.03189156576991081,
  0.03072124347090721,
  0.030254140496253967,
  0.025439897552132607,
  0.02327977865934372,
  0.02171889878809452,
  0.022171877324581146,
  0.018940040841698647,
  0.019433731213212013,
  0.017908746376633644,
  0.016966160386800766,
  0.015527212992310524,
  0.015221275389194489,
  0.014591570012271404,
  0.014356499537825584,
  0.014737877063453197,
  0.014422622509300709,
  0.012234168127179146,
  0.011467145755887032,
  0.011642768047749996,
  0.011849184520542622,
  0.011742861941456795,
  0.010600107721984386,
  0.010745557025074959,
  0.009876968339085579,
  0.010242185555398464,
  0.009259686805307865,
  0.009340941905975342,
  0.008953485637903214,
  0.008568426594138145,
  0.008489527739584446,
  0.008090049028396606,
  0.008564370684325695,
  0.007914789021015167,
  0.00785832293331623],
 'accuracy': [0.2708333432674408,
  0.6465773582458496,
  0.84375,
  0.9308035969734192,
  0.96875,
  0.984375,
  0.9910714030265808,
  0.9985119104385376,
  0.9977678656578064,
  0.9985119104385376,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9992559552192688,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9992559552192688,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'val_loss': [2.297776222229004,
  2.3119757175445557,
  2.3263931274414062,
  2.33884596824646,
  2.3609883785247803,
  2.3691346645355225,
  2.39619517326355,
  2.405829668045044,
  2.3987884521484375,
  2.398494243621826,
  2.354694366455078,
  2.255984306335449,
  2.194587469100952,
  2.0175061225891113,
  1.8293222188949585,
  1.6295416355133057,
  1.4415950775146484,
  1.250285029411316,
  1.0807347297668457,
  0.925035834312439,
  0.8065176010131836,
  0.7027422189712524,
  0.6132355332374573,
  0.5532410144805908,
  0.5074681639671326,
  0.4660699665546417,
  0.4366154074668884,
  0.4166094958782196,
  0.4001880884170532,
  0.39152565598487854,
  0.3801526129245758,
  0.37282073497772217,
  0.3677658438682556,
  0.3619527518749237,
  0.3589867353439331,
  0.3542906641960144,
  0.3516950309276581,
  0.349283903837204,
  0.34885433316230774,
  0.3464423716068268,
  0.3471611738204956,
  0.34285902976989746,
  0.3375314772129059,
  0.3394991457462311,
  0.34027066826820374,
  0.33844292163848877,
  0.33629077672958374,
  0.3352868854999542,
  0.3327336311340332,
  0.3327057361602783],
 'val_accuracy': [0.09799554198980331,
  0.0935412049293518,
  0.13363029062747955,
  0.18040089309215546,
  0.18930958211421967,
  0.1781737208366394,
  0.19599108397960663,
  0.2071269452571869,
  0.24721603095531464,
  0.26503339409828186,
  0.2783964276313782,
  0.2962138056755066,
  0.3184855282306671,
  0.40534520149230957,
  0.4498886466026306,
  0.5011135935783386,
  0.5590200424194336,
  0.6057906746864319,
  0.6636971235275269,
  0.7060133814811707,
  0.7527839541435242,
  0.8062360882759094,
  0.8262805938720703,
  0.8530066609382629,
  0.8619153499603271,
  0.8775055408477783,
  0.8886414170265198,
  0.897550106048584,
  0.8953229188919067,
  0.8997772932052612,
  0.9020044803619385,
  0.904231607913971,
  0.9064587950706482,
  0.9086859822273254,
  0.9064587950706482,
  0.9086859822273254,
  0.9064587950706482,
  0.9086859822273254,
  0.9064587950706482,
  0.9086859822273254,
  0.9064587950706482,
  0.9131402969360352,
  0.9109131693840027,
  0.9064587950706482,
  0.9086859822273254,
  0.9064587950706482,
  0.904231607913971,
  0.9064587950706482,
  0.9131402969360352,
  0.9086859822273254]}
  